In [1]:
import polars as pl

In [2]:
# Import incidents and format columns
df = pl.read_csv(
    "./data/incident_dump.csv",
).with_columns(
    pl.col("reported")
    .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%z")
    .dt.convert_time_zone("America/Chicago"),
    pl.col("reported_date").str.to_date("%Y-%m-%d"),
    pl.col("validated_location").str.split(",").cast(pl.List(pl.Float64)),
)

In [3]:
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""Theft""",930
"""Information""",618
"""Found Property…",567
"""Information / …",378
"""Medical Call""",287
"""Battery""",273
"""Lost Property""",257
"""Information / …",228
"""Mental Health …",220


In [4]:
excluded_list = [
    "Medical Call",
    "Lost Property",
    "Stalking",
    "Sexual Assault",
    "Dating",
    "Stalking",
    "Domestic",
    "Sex",
    "Found Property",
    "Mental Health",
    "Harassment by Electronic Means",
    "Well-Being Check",
    "Threatening Phone Call",
]
df = df.filter(~pl.col("incident").str.contains("|".join(excluded_list)))
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""Theft""",930
"""Information""",618
"""Information / …",378
"""Battery""",273
"""Information / …",228
"""Liquor Law Vio…",196
"""Found Narcotic…",155
"""Information / …",150
"""Information / …",120
